In [1]:
import ape
import pickle
from ape import chain, networks, Contract, project
import pandas as pd
import json

In [2]:
networks.parse_network_choice("ethereum:mainnet:geth").__enter__()

<geth chain_id=1>

In [3]:
with open("./../out/TimelockController.sol/TimelockController.json", "rb") as f:
    data = json.load(f)
timelockType = ape.contracts.base.ContractType(contractName="TimeLockController", abi=data["abi"])
lock_base = networks.compiler_manager.create_contract_container(timelockType)

safe_emerg = Contract("0x23f8008159C0427458b948c3DD7795c6DBE8236F")
safe_base = networks.compiler_manager.create_contract_container(safe_emerg.contract_type)

In [4]:
def toCheckSum(addr):
     return safe_base.provider._web3.toChecksumAddress(addr)

In [5]:
multisigs = {
    "pause": "0x23f8008159C0427458b948c3DD7795c6DBE8236F",
    "maintenance": "0x4a691155686b49f474f745ef4ece4790c9570635",
    "emergency": "0x6d68d53c3d9ad8097e420e2363898701841ee9b2",
    "schedule": "0x76ecc1e721f62d23dbf2132facc4b1b318d2e3c4",
    "resume": "0xdb3daf52b3bfba877904666651357b976c47f635"
}

timelocks = {
    "timelock": "0x7580708993de7ca120e957a62f26a5ddd4b3d8ac",
    "controller": "0x975ab64f4901af5f0c96636dea0b9de3419d0c2f"
}

addrNames = {}

for tl in timelocks:
    addrNames[toCheckSum(timelocks[tl])] = tl

for ms in multisigs:
    addrNames[toCheckSum(multisigs[ms])] = ms


In [13]:
def getName(addr):
    if addr in addrNames:
        return addrNames[addr]
    else:
        addr

In [7]:
def flatten(logs):
    flat = []
    for log in logs:
        t = log.event_arguments
        t["block_number"] = log.block_number
        t["transaction_hash"] = log.transaction_hash.hex()
        flat.append(t)
    return pd.DataFrame(flat)

In [8]:
locks = {}
safes = {}

for tl in timelocks:
    locks[tl] = lock_base.at(toCheckSum(timelocks[tl]))

for ms in multisigs:
    safes[ms] = safe_base.at(toCheckSum(multisigs[ms]))

# Get Signers of Safes

In [9]:
for name in safes:
    safe = safes[name]
    threshold = safe.getThreshold()
    owners = safe.getOwners()
    print("{0} multisig ({1}/{2}):".format(name, threshold, len(owners)))
    for owner in owners:
        print("\t{0}".format(owner))

pause multisig (2/15):
	0x7345B1F517E8C73F283807CF0f932E801f9586E4
	0x332f0c737232D42f6eDA59CB29dd133bA249F001
	0x107931F48357508198E190c2466Dc2978aaa4aB0
	0x957917c4A958e1D6F00d6FEF54b4C400d57d0208
	0x1e11a0A62b88C0c78361280bd3BBF5cCcC0B0C93
	0xcA8827C1fb9D8303941C2cec5Ae5cF6dBB3ef07a
	0x77b49dc62E2Bbbf6aad42ee0AC5B5eeDd7883bB2
	0xFEE6D8EebC4330c51ef37bc5779c9490F66cFa85
	0x0C0C742154aD3632E75dD011C37EDe0AF677A543
	0x39688E9B290096960604422DA6556BD1C521659B
	0x483Bbf9431681A13b6A65e3D161Bd2fF609Aca48
	0x53495Acd57ED93089149ae7D96b9899b710C1F9e
	0x7fb9f93Cc6614dDd76c893EC8b5310674aC3Fc5f
	0x1D93fE338A035f297819EA867275661a4f5B4fdD
	0xDFca360F22b7f3aA13270A9cDeb09d485294a027
maintenance multisig (1/1):
	0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
emergency multisig (1/1):
	0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
schedule multisig (1/1):
	0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
resume multisig (1/1):
	0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266


# Locks

In [10]:
a = locks["controller"]
roleHashes = [a.TIMELOCK_ADMIN_ROLE(), a.CANCELLER_ROLE(), a.DEFAULT_ADMIN_ROLE(), a.EXECUTOR_ROLE(), a.PROPOSER_ROLE() ]

In [11]:
roleToName = {}
roleToName[locks["controller"].CANCELLER_ROLE()] = "CANCELLER_ROLE"
roleToName[locks["controller"].TIMELOCK_ADMIN_ROLE()] = "TIMELOCK_ADMIN_ROLE"
roleToName[locks["controller"].DEFAULT_ADMIN_ROLE()] = "DEFAULT_ADMIN_ROLE"
roleToName[locks["controller"].EXECUTOR_ROLE()] = "EXECUTOR_ROLE"
roleToName[locks["controller"].PROPOSER_ROLE()] = "PROPOSER_ROLE"

In [14]:
for tl in timelocks:
    timelock = locks[tl]
    print("Roles on {0} ({1})".format(tl, timelock.address))

    grantRoleEvents = flatten(
        timelock.RoleGranted.range(
            start_or_stop=15396000,
            stop = chain.blocks.height,
            block_page_size = 10000
        )
    )
    grantRoleEvents["role"] = [roleToName[ape.utils.misc.HexBytes(a)] for a in grantRoleEvents["role"]] 

    for candidate in grantRoleEvents["account"].unique():
        gotRoles = []
        for rh in roleHashes:
            if timelock.hasRole(rh, candidate):
                gotRoles.append(roleToName[rh])

        if len(gotRoles) > 0:
            print("  {0} has {1}".format(getName(candidate), ", ".join(gotRoles)))
    print("")

Roles on timelock (0x7580708993de7CA120E957A62f26A5dDD4b3D8aC)
  timelock has TIMELOCK_ADMIN_ROLE
  schedule has CANCELLER_ROLE, EXECUTOR_ROLE, PROPOSER_ROLE
  emergency has TIMELOCK_ADMIN_ROLE, CANCELLER_ROLE, EXECUTOR_ROLE, PROPOSER_ROLE
  maintenance has EXECUTOR_ROLE

Roles on controller (0x975Ab64F4901Af5f0C96636deA0b9de3419D0c2F)
  controller has TIMELOCK_ADMIN_ROLE
  timelock has TIMELOCK_ADMIN_ROLE, CANCELLER_ROLE, EXECUTOR_ROLE, PROPOSER_ROLE
  emergency has TIMELOCK_ADMIN_ROLE, CANCELLER_ROLE, EXECUTOR_ROLE, PROPOSER_ROLE
  maintenance has EXECUTOR_ROLE

